In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
# logging 
import logging
import logging.config

# allow loading modules from strings
import importlib

# shell utilities for manpiulating files
from shutil import copyfile

# parse configuration files
import configparser

# parse command line arguments
import argparse

# handle file paths in a sane way
from pathlib import Path

In [4]:
# # this works best as a global variable
# # FIXME - best practice for specifying this?
# logConfig = Path('./logging.cfg')
# logging.config.fileConfig(logConfig.absolute())
# # logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

# logger = logging.getLogger(__name__)

In [5]:
def arg_parse():
    '''parse arguments for this script
    
    Arguments parsed:
        -c: configuration file
    
    Returns:
        :obj:argpars.ArgumentParser'''
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
       
    parser.add_argument('-q', type=str, required=False,
                       help='this is for testing only')
    
    args, unknown = parser.parse_known_args()
    logging.info(f'discarding unknwon commandline arguments: {unknown}')
    
    return args

In [38]:
def read_config(cfgfile=None, default=None):
    '''read `cfgfile` file and optionally create one if it does not exist

    Args:
        cfgfile(str): path to configuration file to be read
        default(str): path to default configuration file that should be 
            used if cfgfile does not exist -- this is useful for creating
            config files on first run
    
    Returns:
        obj:configparser.ConfigParser
    '''
    if not cfgfile:
        return {}
    
    cfgfile = Path(cfgfile).expanduser().resolve()
    default = Path(default).expanduser().resolve() if default else None
    
    # check if the path exists
    logging.debug(f'creating parent directory (if needed): {cfgfile.parent}')
    Path(cfgfile.parent).mkdir(parents=True, exist_ok=True)

    # check if specified file exists, otherwise copy the default (if provided)    
    if not cfgfile.exists() and default:
        logging.debug(f'copying {default} to {cfgfile}')
        try:
            copyfile(default, cfgfile)
        except (FileNotFoundError, PermissionError) as e:
            print(f'failed to copy default file ({default}) to specified file: {e}')
            return False
#     elif not default:
#         raise FileNotFoundError(f'could not open {cfgfile}')
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    return config

In [84]:
def config_2dict(configuration):
    
    for section in configuration.sections():
        d[section] = {}
        for opt in configuration.options(section):
            d[section][opt] = myC.get(section, opt)
    
    return d

In [101]:
def main():
    # constants
    waveshare = 'waveshare_epd.' # note trailing '.' !!
    layouts = 'epdlib.layouts'
       
    
    
    # this works best as a global variable
    # FIXME - best practice for specifying this?
    logingConfig = Path('./logging.cfg').resolve()
    logging.config.fileConfig(logingConfig)
    # logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')
    logger = logging.getLogger(__name__)
    
    
    # Configuration Variables
    
    # execute the program - use for checking configuration state 
    execute = True
    # name
    appShortName = 'slimpi'
    name = 'com.txoof.'
    appLongName = name+appShortName        
    
    # Default configuration file if none is specified
    builtin_cfg = Path('./slimpi.cfg')
    # standard location for stored user configuration
    user_cfg = Path(f'~/.config/{appLongName}/slimpi.cfg').expanduser()  
    
    # parse command line arguments
    args = arg_parse()
    
    # use a specified configuration file
    if args.c:
        user_cfg = args.c
        
    logging.info(f'using configuration file: {user_cfg}')        
    
    # read configuration file
    # config parser object
    configuration = read_config(cfgfile=user_cfg, default=builtin_cfg)
    execute = True if configuration else False
    # dictionary
    cfg = config_2dict(configuration)
    
    
    # get all the configuration variables 
    try:
        # lms server settings
        lms_server = cfg['lms_server']
        
        # set epd type
        epd = importlib.import_module('waveshare_epd.'+cfg['layouts']['display'])
        
        playing_layout = importlib.import_module(layouts)
        
        
    except (KeyError, NoSectionError) as e:
        logging.error(f'Error configuration file missing section: {e}')
        execute = False
    
    # if execute was set false anywhere, bail out
    if not execute:
        print('Exiting due to previous errors')
        return
    
        
        
        
    # check if there are any 
#     if len(configuration.sections()) < 1:
# #         raise EOFError(f'config file, {cfg_file}, contained no configuration')
#         print(f'Confuration file: {cfg_file}, contained no configuration')
#         print('exiting')
#         return None
    
    
    
    
    
    
    
    
    
    return configuration

In [102]:
if __name__ == '__main__':
    foo = main()
print(foo)

<ipython-input-5-b795292e2218>:arg_parse:19:INFO - discarding unknwon commandline arguments: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-a6bf9f2b-109f-4d75-8c20-7461b4e9b2cd.json']
<ipython-input-101-b08a88921107>:main:31:INFO - using configuration file: /home/pi/.config/com.txoof.slimpi/slimpi.cfg
<ipython-input-38-82fb72048f05>:read_config:20:DEBUG - creating parent directory (if needed): /home/pi/.config/com.txoof.slimpi
<ipython-input-38-82fb72048f05>:read_config:35:INFO - reading config file /home/pi/.config/com.txoof.slimpi/slimpi.cfg


In [108]:
myL = importlib.import_module('epdlib.layouts')
dir(myL)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__init__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'clock',
 'threeRow',
 'twoColumn']

In [109]:
myL.threeRow

{'title': {'image': None,
  'max_lines': 3,
  'padding': 10,
  'width': 1,
  'height': 0.5714285714285714,
  'abs_coordinates': (0, 0),
  'hcenter': True,
  'vcenter': True,
  'relative': False,
  'font': None,
  'fontsize': None},
 'coverart': {'image': True,
  'max_lines': None,
  'padding': 10,
  'width': 0.4,
  'height': 0.42857142857142855,
  'abs_coordinates': (0, None),
  'hcenter': True,
  'vcenter': True,
  'relative': ['coverart', 'title'],
  'font': None,
  'fontsize': None},
 'artist': {'image': None,
  'max_lines': 2,
  'padding': 10,
  'width': 0.6,
  'height': 0.21428571428571427,
  'abs_coordinates': (None, None),
  'hcenter': False,
  'vcenter': True,
  'relative': ['coverart', 'title'],
  'font': None,
  'fontsize': None},
 'album': {'image': None,
  'max_lines': 2,
  'padding': 10,
  'width': 0.6,
  'height': 0.14285714285714285,
  'abs_coordinates': (None, None),
  'hcenter': False,
  'vcenter': True,
  'relative': ['coverart', 'artist'],
  'font': None,
  'fontsize

In [ ]:
import sys
sys.argv.append('-c')
sys.argv.append('~/slimpi.ini')

In [ ]:
sys.argv
